# 모의고사 1회 문제
## 제 1유형

주어지는 데이터는 153일 동안 공기의 질을 측정한 데이터 셋이다.


In [3]:
# 1. airquality 데이터에 대해서 결측치가 가장 많은 변수를 찾아서 해당 결측치를 0으로 대치하고, 결측치를 제외한 평균과 0으로 대치한 후 평균과의 차이를 구하시오.

import pandas as pd
import numpy as np

air = pd.read_csv("https://raw.githubusercontent.com/YoungjinBD/dataset/main/airquality.csv")
air.head(5)

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2
2,12.0,149.0,12.6,74,5,3
3,18.0,313.0,11.5,62,5,4
4,NaN,NaN,14.3,56,5,5


In [4]:
air.isnull().sum() # Ozone 컬럼의 데이터에 결측치가 가장 많음을 알 수 있다.

,0
Ozone,37
Solar.R,7
Wind,0
Temp,0
Month,0
Day,0


In [ ]:
# 결측치를 0으로 채우기 전 평균을 구한다.
mean_nan = float(air['Ozone'].mean())
mean_nan

42.12931034482759

In [ ]:
air['Ozone'].fillna(0,inplace=True)  # inplace=True 는 데이터프레임 자체를 수정하겠다는 소리
air.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Ozone    153 non-null    float64
 1   Solar.R  146 non-null    float64
 2   Wind     153 non-null    float64
 3   Temp     153 non-null    int64  
 4   Month    153 non-null    int64  
 5   Day      153 non-null    int64  
dtypes: float64(3), int64(3)
memory usage: 7.3 KB


In [ ]:
# 결측치를 채우고 나서의 평균을 구한다.
mean_full = float(air['Ozone'].mean())
mean_full

31.941176470588236

In [ ]:
# 둘의 평균의 차이를 구한다.
print(abs(mean_full-mean_nan))

10.188133874239352


In [ ]:
# 2. Wind변수에 대해서 Min-Max 정규화를 진행한 후 평균값과 Z정규화 이후 평균 값의 차이를 구하시오.

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

MM_scaler = MinMaxScaler()
S_scaler = StandardScaler()

air['MM'] = MM_scaler.fit_transform(air[['Wind']]) # fit_transform 은 2차원 배열의 요구
air['S'] = S_scaler.fit_transform(air[['Wind']])

print(abs(air["MM"].mean() - air['S'].mean()))

0.4346061231510147


In [ ]:
# 다른 정답
air['MMM'] = (air['Wind']-air['Wind'].min())/(air['Wind'].max()-air['Wind'].min())
air['SS'] = (air['Wind']-air['Wind'].mean())/air['Wind'].std()

print(abs(air['MMM'].mean()-air["SS"].mean()))

0.43460612315101477


In [ ]:
# 3. 월별(5~9) 평균기온을 구하시오.
air.groupby("Month")["Temp"].mean()

,Temp
Month,
5,65.548387
6,79.100000
7,83.903226
8,83.967742
9,76.900000


## 제 2유형

은행에서 수집한 고객 5,000명의 금융정보에 따른 대출여부가 들어있는 참조 데이터를 이용하여 대출여부를 분류하는 최적의 이웃의 크기값(k)을 구하고, 이때 분류정확도를 산출하시오.

(단. 참조데이터는 7:3의 비율로 트레이닝 데이터와 테스트 데이터로 구분하고, 트레이닝 데이터와 테스트 데이터의 대출여부(y)의 비율도 유지한다. 또한, Nomalizer를 사용하여 스케일링을 진행한다.)

In [ ]:
Bank_data = pd.read_csv("https://raw.githubusercontent.com/YoungjinBD/dataset/main/Bank_Personal_Loan_Modeling.csv")
Bank_data

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,0,1,0
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,0,1,0
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,0,1,0


In [ ]:
# 데이터 전처리, 상관관계분석
corr = Bank_data.corr(method= 'pearson')
corr

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
ID,1.000000,-0.008473,-0.008326,-0.017695,0.013432,-0.016797,-0.024675,0.021463,-0.013920,-0.024801,-0.016972,-0.006909,-0.002528,0.017028
Age,-0.008473,1.000000,0.994215,-0.055269,-0.029216,-0.046418,-0.052012,0.041334,-0.012539,-0.007726,-0.000436,0.008043,0.013702,0.007681
Experience,-0.008326,0.994215,1.000000,-0.046574,-0.028626,-0.052563,-0.050077,0.013152,-0.010582,-0.007413,-0.001232,0.010353,0.013898,0.008967
Income,-0.017695,-0.055269,-0.046574,1.000000,-0.016410,-0.157501,0.645984,-0.187524,0.206806,0.502462,-0.002616,0.169738,0.014206,-0.002385
ZIP Code,0.013432,-0.029216,-0.028626,-0.016410,1.000000,0.011778,-0.004061,-0.017377,0.007383,0.000107,0.004704,0.019972,0.016990,0.007691
Family,-0.016797,-0.046418,-0.052563,-0.157501,0.011778,1.000000,-0.109275,0.064929,-0.020445,0.061367,0.019994,0.014110,0.010354,0.011588
CCAvg,-0.024675,-0.052012,-0.050077,0.645984,-0.004061,-0.109275,1.000000,-0.136124,0.109905,0.366889,0.015086,0.136534,-0.003611,-0.006689
Education,0.021463,0.041334,0.013152,-0.187524,-0.017377,0.064929,-0.136124,1.000000,-0.033327,0.136722,-0.010812,0.013934,-0.015004,-0.011014
Mortgage,-0.013920,-0.012539,-0.010582,0.206806,0.007383,-0.020445,0.109905,-0.033327,1.000000,0.142095,-0.005411,0.089311,-0.005995,-0.007231
Personal Loan,-0.024801,-0.007726,-0.007413,0.502462,0.000107,0.061367,0.366889,0.136722,0.142095,1.000000,0.021954,0.316355,0.006278,0.002802


In [ ]:
Bank_data.columns

Index(['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Personal Loan', 'Securities Account',
       'CD Account', 'Online', 'CreditCard'],
      dtype='object')

In [ ]:
# 여기서 0.1의 상관관계 수치를 넘지 못하는 것들을 제거
Bank_data.drop(columns=['ID', 'Age', 'Experience','ZIP Code','Securities Account','Online', 'CreditCard'],inplace=True)

,Income,Family,CCAvg,Education,Mortgage,Personal Loan,CD Account
0,49,4,1.6,1,0,0,0
1,34,3,1.5,1,0,0,0
2,11,1,1.0,1,0,0,0
3,100,1,2.7,2,0,0,0
4,45,4,1.0,2,0,0,0
...,...,...,...,...,...,...,...
4995,40,1,1.9,3,0,0,0
4996,15,4,0.4,1,85,0,0
4997,24,2,0.3,3,0,0,0
4998,49,3,0.5,2,0,0,0


In [ ]:
Bank_data['Personal Loan'].value_counts()

,count
Personal Loan,
0,4520
1,480


In [ ]:
Bank_data.describe()
col_name = Bank_data.columns

✅ 올바른 순서

```
# 데이터 분리
`X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)`

# 데이터 정규화
norm = Normalizer()
X_train = norm.fit_transform(X_train)   # 훈련 데이터 기준으로 맞춤
X_test = norm.transform(X_test)         # 훈련 기준 그대로 적용
```

**데이터를 정규화한 후 나누게 된다면?**

이렇게 하면 테스트 데이터가 훈련 데이터 정보를 미리 반영한 정규화 기준으로 바뀌게 되어,
📉 데이터 누수(data leakage) 발생 → 모델의 성능이 과대평가될 수 있습니다.

In [ ]:
# 데이터 전처리, Normalizer
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split

X = Bank_data.drop(columns=["Personal Loan"])
y = Bank_data['Personal Loan']

# data split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

norm = Normalizer()
X_train = norm.fit_transform(X_train)
X_test = norm.transform(X_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

 # K-분류 model
for i in range(1,25):
  model = KNeighborsClassifier(n_neighbors = i)
  model.fit(X_train,y_train)
  pred = model.predict(X_test)
  print(accuracy_score(y_test,pred)) # k = 2일 때 0.929 성능

0.9206666666666666
0.9293333333333333
0.9186666666666666
0.9206666666666666
0.9173333333333333
0.9233333333333333
0.9193333333333333
0.918
0.918
0.9186666666666666
0.916
0.9133333333333333
0.916
0.9173333333333333
0.914
0.9146666666666666
0.916
0.914
0.9133333333333333
0.914
0.9146666666666666
0.9146666666666666
0.9153333333333333
0.9153333333333333


## 제 3유형

측정한 온도(Temp)의 평균값이 평균온도인 77과 일치하는지 t-검정을 이용하여 확인하시오.

In [ ]:
air = air = pd.read_csv("https://raw.githubusercontent.com/YoungjinBD/dataset/main/airquality.csv")
air.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Ozone    116 non-null    float64
 1   Solar.R  146 non-null    float64
 2   Wind     153 non-null    float64
 3   Temp     153 non-null    int64  
 4   Month    153 non-null    int64  
 5   Day      153 non-null    int64  
dtypes: float64(3), int64(3)
memory usage: 7.3 KB


In [ ]:
# 표본 평균 X를 소수점 둘째자리까지 반올림하여 구하시오.
sample_mean = round(air['Temp'].mean(),2)
sample_mean

np.float64(77.88)

In [ ]:
# 가설 검정을 위한 검정통계량을 소수점 둘째자리까지 반올림하여 구하시오.
# t-검정: 모표본편차, 모분산을 알 수 없을 경우 사용

from scipy import stats

t_statistic,p_value = stats.ttest_1samp(air['Temp'],popmean=77,alternative='two-sided') # sample data, popmean, alternative

round(t_statistic,2)

np.float64(1.15)

In [ ]:
# 통계량에 대한 p_value와 소수점 넷째자리까지 반올림하여 구하시고 유의수준 0.05에서 결과를 논하시오.

p_value = round(p_value,4)

if p_value > 0.05  :
  print('귀무가설을 기각하지못합니다(채택), 77과 일치합니다.')
else:
  print('귀무가설을 기각합니다. 77과 일치하지 않습니다.')

귀무가설을 기각하지못합니다(채택), 77과 일치합니다.


# 모의고사 2회

## 제 1유형

주어진 데이터는 IBM의 근무지에 대한 직무정보와 퇴사 여부를 정리한 것이다.

In [ ]:
# 데이터에서 'Attrition'은 퇴사여부에 대한 종속변수에 해당한다. 종속변수의 값을 다음과 같이 수치로 변환해 새 칼럼으로 추가하고 각 범주별 레코드 수를 계산하시오.

IBM_data = pd.read_csv("https://raw.githubusercontent.com/YoungjinBD/dataset/main/HR-Employee-Attrition.csv")
IBM_data.head(5)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [ ]:
from sklearn.preprocessing import LabelEncoder

Scaler = LabelEncoder()
IBM_data["Att_scaler"] = Scaler.fit_transform(IBM_data['Attrition'])
IBM_data.head(5)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Att_scaler
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,80,0,8,0,1,6,4,0,5,1
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,80,1,10,3,3,10,7,1,7,0
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,80,0,7,3,3,0,0,0,0,1
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,80,0,8,3,3,8,7,3,0,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,80,1,6,3,3,2,2,2,2,0


In [ ]:
IBM_data["Att_scaler"].value_counts()

,count
Att_scaler,
0,1233
1,237


In [ ]:
# 데이터 셋의 데이터 타입별 칼럼 개수를 계산하고, 범주형 변수 중 유이한 값을 1개만 가지고 있는 컬럼(변수)을 찾아내어 그 변수를 데이터에서 제거하시오.

IBM_data.dtypes.value_counts()

,count
int64,27
object,9


In [ ]:
category_list = IBM_data.select_dtypes(include='object').columns
category_list

Index(['Attrition', 'BusinessTravel', 'Department', 'EducationField', 'Gender',
       'JobRole', 'MaritalStatus', 'Over18', 'OverTime'],
      dtype='object')

In [ ]:
for i in category_list:
  print(IBM_data[i].value_counts())
  print('*'*30)

Attrition
No     1233
Yes     237
Name: count, dtype: int64
******************************
BusinessTravel
Travel_Rarely        1043
Travel_Frequently     277
Non-Travel            150
Name: count, dtype: int64
******************************
Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64
******************************
EducationField
Life Sciences       606
Medical             464
Marketing           159
Technical Degree    132
Other                82
Human Resources      27
Name: count, dtype: int64
******************************
Gender
Male      882
Female    588
Name: count, dtype: int64
******************************
JobRole
Sales Executive              326
Research Scientist           292
Laboratory Technician        259
Manufacturing Director       145
Healthcare Representative    131
Manager                      102
Sales Representative          83
Research Director             80
Human Resources      

In [ ]:
IBM_data.drop(columns=['Over18'],inplace= True)

In [ ]:
# 원래 데이터에서 숫자형인 컬럼(변수)만 추출하여 새로운 데이터 프레임을 생성하고, 각 변수간의 상관계수(피어슨 상관계수)가 0.9 이상인 두 개의 컬럼을 찾아내어 그 변수 중
# 하나를 제거하시오.

new_data = IBM_data.select_dtypes(include = 'int64')
corr = new_data.corr()

In [ ]:
for i in range(0,25):
  for j in range(i+1,26):
    if (corr.iloc[i,j] >=  0.9): # iloc[행번호, 열번호], loc[행이름,열이름]
      print(i,j,corr.iloc[i,j])

9 11 0.9502999134798473


In [ ]:
new_data.columns[9]

'JobLevel'

In [ ]:
new_data.drop(columns=['JobLevel'],inplace=True)

## 제 2유형

데이터셋을 이용하여 파킨슨병을 예측하는 모델을 로지스틱회귀모형을 적용하여 생성하고, 이때 파키슨 병을 예측하는데 영향을 미치는 변수를 중요한 순서대로 3개 선정하세요.

이 모델에서 파킨슨병으로 진단하는 기준(threashold or cutoff)을 0.5로 했을 때와 0.8로 했을 때의 F1-score를 비교하고 해석하시오.

<조건>
- 필요없는 칼럼인 'name'을 제거
- 데이터 정규화는 min-max 사용
- 로지스틱 회귀를 위해 상수항을 추가
- Status 는 카테고리 타입으로 변환
- 트레이닝 셋과 테스트 셋의 비율은 9:1
- 모델은 로지스틱 회귀분석 사용
- 모델의 최적화 방법론은 'bfgs'사용

⛔보통의 흐름
전처리-> 데이터 분리-> 정규화-> 상수항추가-> 모델학습

In [ ]:
Parkin_data = pd.read_csv("https://raw.githubusercontent.com/YoungjinBD/dataset/main/Parkinsons.csv")
Parkin_data.head(5)

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [ ]:
Parkin_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              195 non-null    object 
 1   MDVP:Fo(Hz)       195 non-null    float64
 2   MDVP:Fhi(Hz)      195 non-null    float64
 3   MDVP:Flo(Hz)      195 non-null    float64
 4   MDVP:Jitter(%)    195 non-null    float64
 5   MDVP:Jitter(Abs)  195 non-null    float64
 6   MDVP:RAP          195 non-null    float64
 7   MDVP:PPQ          195 non-null    float64
 8   Jitter:DDP        195 non-null    float64
 9   MDVP:Shimmer      195 non-null    float64
 10  MDVP:Shimmer(dB)  195 non-null    float64
 11  Shimmer:APQ3      195 non-null    float64
 12  Shimmer:APQ5      195 non-null    float64
 13  MDVP:APQ          195 non-null    float64
 14  Shimmer:DDA       195 non-null    float64
 15  NHR               195 non-null    float64
 16  HNR               195 non-null    float64
 1

In [ ]:
# 필요없는 컬럼인 name 제거

Parkin_data.drop(columns=["name"],inplace=True)

In [ ]:
# Status 는 카테고리로 변환

Parkin_data['status'] = Parkin_data['status'].astype('category')

⛔ 정규화 과정시에 데이터가 array로 변환하는데 다시 데이터 프레임으로 바꾸고 칼럼명과 인덱스(y와 같게)를 맞춰주고 모델 훈련을 시킨다.

In [ ]:
X = Parkin_data.drop(columns=['status'])
x_col = X.columns
y = Parkin_data['status']

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=x_col, index=y_train.index)
X_test = pd.DataFrame(scaler.transform(X_test), columns=x_col, index=y_test.index)

has_constant 옵션 의미
기본값은 'skip'입니다.

즉, 데이터에 이미 상수항(모든 값이 1인 컬럼)이 있으면 추가하지 않고, 없으면 추가해줍니다.

만약 상수항이 이미 있다고 확신하면 'raise'로 설정해 중복 추가를 방지하거나,

강제로 상수항을 추가하려면 'add'로 설정할 수 있습니다.

In [ ]:
import statsmodels.api as sm

# 상수항을 각각 추가해준다.
X_train = sm.add_constant(X_train,has_constant='add')
X_test = sm.add_constant(X_test,has_constant='add')

model = sm.Logit(y_train,X_train) # 종속변수, 독립변수 순으로 넣어준다.

result = model.fit(method ='bfgs',maxiter=1000) # bfgs: 비선형함수 최적화, 최적화가 수렴할 때 까지 반복:1000, 보통 35 또는 100

result.summary()
# pvalue가 작은게 영향을 많이 미친다.

Optimization terminated successfully.
         Current function value: 0.241374
         Iterations: 284
         Function evaluations: 285
         Gradient evaluations: 285


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 status   No. Observations:                  175
Model:                          Logit   Df Residuals:                      152
Method:                           MLE   Df Model:                           22
Date:                Mon, 16 Jun 2025   Pseudo R-squ.:                  0.5766
Time:                        15:15:34   Log-Likelihood:                -42.240
converged:                       True   LL-Null:                       -99.758
Covariance Type:            nonrobust   LLR p-value:                 1.379e-14
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -6.9611      5.390     -1.292      0.197     -17.525       3.603
MDVP:Fo(Hz)         -0.8913      3.643     -0.245      0.807      -8.031       6.249
MDVP:Fhi(Hz)        -1.0283      2.180     -0.472      0.637      -5.301       3.244
MDVP:Flo(Hz)         0.6661      1.947      0.342      0.732      -3.150       4.482
MDVP:Jitter(%)     -42.9353     35.290     -1.217      0.224    -112.102      26.232
MDVP:Jitter(Abs)     3.3625     23.816      0.141      0.888     -43.316      50.041
MDVP:RAP            39.9810   2637.709      0.015      0.988   -5129.835    5209.797
MDVP:PPQ           -51.8757     37.476     -1.384      0.166    -125.327      21.576
Jitter:DDP          39.8303   2640.123      0.015      0.988   -5134.715    5214.376
MDVP:Shimmer        45.3711    109.209      0.415      0.678    -168.674     259.417
MDVP:Shimmer(dB)    15.1229     39.316      0.385      0.701     -61.936      92.182
Shimmer:APQ3       -20.7432   5822.136     -0.004      0.997   -1.14e+04    1.14e+04
Shimmer:APQ5       -19.4819     32.988     -0.591      0.555     -84.137      45.173
MDVP:APQ            36.3016     49.350      0.736      0.462     -60.423     133.026
Shimmer:DDA        -21.1490   5828.436     -0.004      0.997   -1.14e+04    1.14e+04
NHR                  9.4703     15.127      0.626      0.531     -20.178      39.118
HNR                  1.0162      5.188      0.196      0.845      -9.152      11.184
RPDE                -2.2069      2.149     -1.027      0.304      -6.419       2.005
DFA                  2.2965      2.314      0.993      0.321      -2.238       6.831
spread1             -5.3063      8.232     -0.645      0.519     -21.440      10.827
spread2              5.1215      2.798      1.830      0.067      -0.363      10.606
D2                   3.0104      3.159      0.953      0.341      -3.182       9.202
PPE                 23.1683     11.730      1.975      0.048       0.178      46.159
====================================================================================

Possibly complete quasi-separation: A fraction 0.13 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [ ]:
# 임계치에 따른 모델 정확도 분류
# 로지스틱 회귀분석은 0~1사이의 확률값을 나타냄

def cut_off(y,threshold):
  Y = y.copy()
  Y[Y > threshold] = 1
  Y[Y <= threshold] = 0 # 원래 float
  return(Y.astype(int))

pred = result.predict(X_test)
pred_8 = cut_off(pred,0.8)
pred_5 = cut_off(pred,0.5)
print(pred_8,'\n', pred_5)

116    1
94     1
8      1
19     1
102    1
70     1
170    0
55     1
31     0
124    0
53     1
112    1
132    1
39     1
113    1
154    1
58     1
136    1
22     1
81     1
dtype: int64 
 116    1
94     1
8      1
19     1
102    1
70     1
170    0
55     1
31     0
124    0
53     1
112    1
132    1
39     1
113    1
154    1
58     1
136    1
22     1
81     1
dtype: int64


In [ ]:
# f1-score 산출

from sklearn.metrics import f1_score

print(f1_score(y_test,pred_8)) # 실제값, 예측값의 순서
print(f1_score(y_test,pred_5))

0.9411764705882353
0.9411764705882353


## 제 3유형

결혼유무에 따라 초과근로 여부에 차이가 있는지 카이제곱 검정을 이용하여 확인하시오.

In [ ]:
# 주어진 데이터로 결혼유무와 초과근로 간의 분할표를 만들고, 결혼한 집단의 초과근로자 수와 초과근로 하지 않은 자 수의 차이를 정수로 나타내시오.

married_data = pd.read_csv("https://raw.githubusercontent.com/YoungjinBD/dataset/main/HR-Employee-Attrition.csv")
married_data.head(5)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [ ]:
married_data.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

In [ ]:
table = pd.crosstab(married_data["MaritalStatus"],married_data["OverTime"])
abs(487-186)

301

In [ ]:
# 가설 검정을 위한 검정통계량을 소수점 둘째자리까지 반올림하여 구하시오
from scipy.stats import chi2_contingency

x_statistic,p_value,df,excepted = chi2_contingency(table)
round(x_statistic,2)

np.float64(0.82)

In [ ]:
# 통계량에 대한 p_value값을 소수점 넷째자리까지 반올림하여 구하시고, 유의수준 0.05 내에서 결과를 논하시오.

p_value = round(p_value,4)
if p_value > 0.05:
  print('귀무가설을 기각하지 못합니다, 채택합니다. 즉, 결혼유무와 초과근무는 연관성이 없을 가능성이 있습니다.')
else:
  print('귀무가설을 기각합니다, 즉 결혼유무와 초과근무의 연관성이 있을 가능성이 있습니다.')

귀무가설을 기각하지 못합니다, 채택합니다. 즉, 결혼유무와 초과근무는 연관성이 없을 가능성이 있습니다.
